In [2]:
from qiskit.opflow import X, Z, Y, I
from qiskit.algorithms.optimizers import ADAM, CG, COBYLA, L_BFGS_B, GradientDescent, NELDER_MEAD, \
                                            NFT, POWELL, SLSQP, SPSA, TNC

In [3]:
#Builds a Pauli term by defining which pauli you want at what ndx and the size of the term
def buildMonomial(pauli, ndx, size):
    result = pauli if ndx == 0 else I
    for i in range(1, size):
        result ^= pauli if i == ndx else I
    return result

def buildTerm(size, pauli, coeff = 1.0):
    result = pauli.get(0) or I
    for i in range(1, size):
        result ^= pauli.get(i) or I
    return result * coeff
    
#converts Coupling Matrix -> PauliSumOp
def buildHamiltonian(coupling_mat):
    result = None
    paulis = [X, Y, Z]
    size = len(coupling_mat) // 3
    
    dim = len(coupling_mat.shape)
    num_entries = np.prod(coupling_mat.shape)
    
    for i in range(num_entries):
        f = lambda i,j : i // ((size*3)**j)
        loc = tuple([(f(i,j) % (size*3)) for j in range(dim)])
        
        if coupling_mat[loc] != 0 and tuple(np.sort(loc)) == loc:
            pauli_map = [paulis[aisle // size] for aisle in loc]
            ndx_map = [aisle % size for aisle in loc]
            ndx_pauli = dict(zip(ndx_map, pauli_map))
            term = buildTerm(size, ndx_pauli, coupling_mat[loc])
            result = result + term if result else term
    return result

In [4]:
# Wrapper for Coupling Matrix to make some things easier
# Values are not split across diagonal
class CouplingMatrix:
    
    def __init__(self, N):
        self.qubits = N
        self.matrix = np.zeros((3*N, 3*N), dtype=float)
        self.set_submatrices()
        self.constant = 0
    
    def __setitem__(self, key, value):
        self.matrix[key] = value
        self.matrix[key[::-1]] = value
        self.set_submatrices()
        
    def __getitem__(self, key):
        return self.matrix[key]
    
    def __str__(self):
        return self.matrix.__str__()
    
    def __repr__(self):
        return self.matrix.__str__()
    
    def getHamiltonian(self):
        hamiltonian = buildHamiltonian(self.matrix)
        if hamiltonian:
            return hamiltonian
    
    def setConstant(self, value):
        self.constant = value
    
    def constant(self):
        return self.constant
        
    def set_submatrices(self):
        self.X = self.matrix[:N,:N]
        self.Y = self.matrix[N:2*N,N:2*N]
        self.Z = self.matrix[2*N:,2*N:]

In [5]:
all_optimizers = [ADAM, CG, COBYLA, L_BFGS_B, GradientDescent, NELDER_MEAD, NFT, POWELL, SLSQP, SPSA, TNC]
def setOptimizers(optimizers):
    all_optimizers = [ADAM, CG, COBYLA, L_BFGS_B, GradientDescent, \
                  NELDER_MEAD, NFT, POWELL, SLSQP, SPSA, TNC]
    
    bool_optimizers = [o in optimizers for o in all_optimizers]
    
    for i, optimizer in enumerate(all_optimizers):
        if bool_optimizers[i]:
            print(f"\x1b[32m{optimizer.__name__:15} ON")
        else: 
            print(f"\x1b[31m{optimizer.__name__:15} OFF")
            
    optimizers = [o(maxiter=max_iters) for o in optimizers]
        
    return optimizers